In [264]:
import pandas as pd
import json
from ast import literal_eval

In [283]:
org = pd.read_csv('../../data/train.csv')

In [265]:
def cut(text):
    st = text.find('"')+1
    text = text[st:]
    ed = text.find('"')
    text = text[:ed]
    return text

In [266]:
data_list = []
for i in range(4):
    data_list.append(pd.read_csv(f'data_filter_{i}.csv'))
df = pd.concat(data_list)

In [267]:
df['fname_list'] = df['fname_list'].apply(literal_eval)

In [268]:
df = df[~df['output'].isna()]

In [269]:
complete_1 = []
no_len_matchs = []
for fnames,output in df[['fname_list','output']].values:
    try:
        outputs = json.loads(output.replace("json","").replace("```",""))
        if len(outputs) == len(fnames):
            for i1,i2 in zip(outputs,fnames):
                complete_1.append({'fname':i2,'output':i1['판단']})
        else:
            no_len_matchs.append({'fname_list':fnames,'output':output})

    except:
        no_len_matchs.append({'fname_list':fnames,'output':output})
print(len(complete_1))
print(len(no_len_matchs))

6360
2031


In [276]:


def output_cut(output,keyword):
    outputs = [cut(i.split(keyword)[1]) for i in output.split('\n') if i.find(keyword) != -1]
    if len(outputs) == 1:
        outputs = [cut(i.split(keyword)[1]) for i in output.split(',') if i.find(keyword) != -1]
    return outputs

complete_2 = []
bad = []
c1 = 0
c2 = 0
keywords = ['"판단":','"판단 결과":','"대화_요약_일치도":','"결과":','"일치_여부":',
'"judgment":','"결론":','"대화_요약_일치여부":','"평가":','"result":','"대화_요약_판단":',
'"판단결과":','"평가":','"요약":','"대화_요약_상태":','"일치 여부":','"대화_요약_일치_여부":',
'"판단_결과":','"대화_요약_일치성":','"일치":','"요약_평가":','"평가 결과":','"요약 평가":',
'"대화_요약_평가":','"판단 결과":','"판단_이유":','"요약_판단":','"판단 이유":','"대화_요약_관계":',
'"summary_judgment":','"판단 결과":','"대화_요약_판정":','"대화_요약_일치 여부":','"결정":',
'"대화_요약_결과":','"일치여부":','"요약_정확성":','"대화 요약 평가":']

for no_len_match in no_len_matchs:
    output = no_len_match['output']
    fnames = no_len_match['fname_list']
    if isinstance(output,str):
        outputs = []
        for keyword in keywords:
            outputs = output_cut(output,keyword)
            if len(outputs) > 0:
                break
        if len(fnames) == len(outputs):
            for i1,i2 in zip(outputs,fnames):
                complete_2.append({'fname':i2,'output':i1})
        else:
            c1+=1
            bad.append(no_len_match)
    else:
        c2+=1
        bad.append(no_len_match)

print('complete_2:',len(complete_2))
print('bad:',len(bad))
print('c1:',c1)
print('c2:',c2)


complete_2: 6079
bad: 4
c1: 4
c2: 0


In [279]:
dfs = pd.DataFrame(complete_1+complete_2)
print(dfs.shape)

dfs = dfs[dfs['output'].isin(['일치','불일치','부분적 일치'])]
print(dfs.shape)

(12439, 2)
(12347, 2)


In [ ]:

a = dfs.value_counts('output').reset_index(name='count')
b = dfs.value_counts('output',normalize=True).reset_index(name='%')
pd.merge(a,b,on='output',how='left')

,output,count,%
0,일치,8130,0.65846
1,부분적 일치,3783,0.30639
2,불일치,434,0.03515


In [284]:
dfs.value_counts('output',normalize=True)

output
일치        0.65846
부분적 일치    0.30639
불일치       0.03515
Name: proportion, dtype: float64

In [294]:
dfs[dfs['fname'].isin(['일치'])]

,fname,output


In [297]:
import os
import shutil



os.makedirs('../../create_data_v5_A/',exist_ok=True)
a = org[org['fname'].isin(dfs[dfs['fname'].isin(['일치'])])]
org[org['fname'].isin(dfs[dfs['output'].isin(['일치'])]['fname'])].to_csv('../../create_data_v5_A/train.csv',index=False)
shutil.copyfile('../../data/test.csv','../../create_data_v5_A/test.csv')
shutil.copyfile('../../data/dev.csv','../../create_data_v5_A/dev.csv')

'../../create_data_v5_A/dev.csv'

In [298]:
import os
import shutil
os.makedirs('../../create_data_v5_B/',exist_ok=True)
org[org['fname'].isin(dfs[dfs['output'].isin(['일치','부분적 일치'])]['fname'])].to_csv('../../create_data_v5_B/train.csv',index=False)
shutil.copyfile('../../data/test.csv','../../create_data_v5_B/test.csv')
shutil.copyfile('../../data/dev.csv','../../create_data_v5_B/dev.csv')

'../../create_data_v5_B/dev.csv'